In [2]:
!pip install sympy

In [7]:
import numpy as np
import pandas as pd
from sclibrary.simplicial_complex import SimplicialComplexNetwork

In [8]:
data_folder = 'data/sample_data'

B1 = pd.read_csv(data_folder + '/B1.csv', header=None)
print(B1.shape)
B1

(5, 7)


,0,1,2,3,4,5,6
0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,-1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,-1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [9]:
B2 = pd.read_csv(data_folder + '/B2.csv', header=None)
print(B2.shape)
B2

(2, 7)


,0,1,2,3,4,5,6
0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,-1.0,0.0,1.0


In [10]:
from sclibrary.read_incidence_matrix import ReadIncidenceMatrix

data = ReadIncidenceMatrix(B1, B2)
data.get_data_summary()
print("Nodes", data.get_nodes())
print("Edges", data.get_edge_list(rank=1))
print("Triangle Edges", data.get_edge_list(rank=2))
print("Adjacency", data.get_adjacency_matrix())

edges = data.get_edge_list()
print(edges)
sc = SimplicialComplexNetwork(edges, pos=None)

number of nodes:  5
number of edges:  7
number of triangles:  2
Nodes [0, 1, 2, 3, 4]
Edges [(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4)]
Triangle Edges [[1, 2, 3], [1, 3, 4]]
Adjacency [[ 0 -1 -1  0  0]
 [ 1  0 -1 -1 -1]
 [ 1  1  0 -1  0]
 [ 0  1  1  0 -1]
 [ 0  1  0  1  0]]
[(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4), [1, 2, 3], [1, 3, 4]]


In [11]:
L1 = sc.hodge_laplacian_matrix(rank=1)
L1L = sc.lower_laplacian_matrix(rank=1)
L1U = sc.upper_laplacian_matrix(rank=1)
L1

array([[ 2.,  1., -1., -1., -1.,  0.,  0.],
       [ 1.,  2.,  1.,  0.,  0., -1.,  0.],
       [-1.,  1.,  3.,  0.,  1.,  0.,  0.],
       [-1.,  0.,  0.,  4.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  3.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  3., -1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  3.]], dtype=float32)

In [12]:
def get_eigendecomposition(hodgle_lap_mat, lower_lap_mat, upper_lap_mat):
    # harmonic eigenvectors
    u_h = get_eigenvectors(hodgle_lap_mat)
    # gradient eigenvectors
    u_g = get_eigenvectors(lower_lap_mat)
    ## curl eigenvectors
    u_c = get_eigenvectors(upper_lap_mat)
    return u_h, u_g, u_c

def get_eigenvectors(lap_mat):
    eigenvalues, eigenvectors = np.linalg.eig(lap_mat)
    lambda_values = np.diag(eigenvalues)
    lambda_values[lambda_values < 1e-3] = 0
    # L(k) = U(k) * lambda(k) * U(k).T
    assert np.allclose(np.rint(eigenvectors @ lambda_values @ np.linalg.inv(eigenvectors)), lap_mat)
    return eigenvectors

In [13]:
u_h, u_g, u_c = get_eigendecomposition(L1, L1L, L1U)
u_g

array([[ 6.1721343e-01, -5.1877373e-01,  2.8867513e-01, -5.0000000e-01,
         1.2879470e-01,  7.2972907e-04, -6.1703771e-01],
       [-6.1721343e-01, -3.0389065e-01,  5.7735026e-01, -6.4459498e-16,
         4.3973261e-01, -7.2972907e-04,  6.1703771e-01],
       [ 3.8575837e-01,  2.1488312e-01,  2.8867513e-01,  5.0000000e-01,
         3.1093791e-01,  8.5687749e-02, -3.7199578e-01],
       [ 1.5430336e-01, -2.1488312e-01,  2.8867513e-01,  5.0000000e-01,
        -3.1093791e-01,  5.1509166e-01, -1.6826583e-01],
       [ 7.7151679e-02, -5.1877373e-01, -2.8867513e-01,  5.0000000e-01,
         1.2879470e-01, -6.0004967e-01, -7.6776087e-02],
       [-2.3145503e-01, -4.2976624e-01, -2.8236944e-16,  1.4359947e-16,
        -6.2187582e-01,  8.4958017e-02,  2.4504194e-01],
       [-7.7151679e-02, -3.0389065e-01, -5.7735026e-01,  9.4069851e-17,
         4.3973261e-01,  6.0004967e-01,  7.6776087e-02]], dtype=float32)

In [250]:
import sympy

def get_matrix_image(matrix):
    matrix_t = sympy.Matrix(matrix.T)
    print(matrix_t)
    rref_matrix, pivot_columns = matrix_t.rref()
    return rref_matrix, pivot_columns

def kernel(x1, x2):
    return x1.T @ x2

def kernel_kenny(a):
    result = []
    for i in range(a.shape[1]):
      current_result = []
      for j in range(a.shape[1]):
        x1 = a[:, i]
        x2 = a[:, j]
        current_result.append(kernel(x1, x2))
    
      result.append(current_result)

    return np.array(result)